# A projectile is shot horizontally at 1000 mph from the coordinates 0° N, 0° E pointed at north, 120000 ft above the surface. At what coordinates will hit the surface?
## assumptions:
- no air resistance
- earth is a perfect sphere with radius 3959 mi
- earth's rotation is exactly 1/24h
- gravitational acceleration is 9.81 m/s² at the surface
- the mass and size of the projectile are negligible

In [1]:
import math

import ipywidgets as widgets
from mpl_toolkits import mplot3d
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import scipy.ndimage

def degrees_to_radians(d):
    return d * math.pi / 180

def radians_to_degress(rad):
    return rad * 180 / math.pi

class PositionCartesian:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

class PositionSpherical:
    def __init__(self, r, θ, φ, time=0):
        self.r = r
        self.θ = θ
        self.φ = φ
        self.time = time
        
    def cartesian(self):
        return PositionCartesian(self.r * math.sin(self.θ) * math.cos(self.φ), self.r * math.sin(self.θ) * math.sin(self.φ),  self.r * math.cos(self.θ))
        
    def __str__(self):
        return f'({self.r}, {self.θ}, {self.φ})'
    
    def earth_fixed_pos(self):
        '''
        Returns an earth-fixed position assuming self is space-fixed
        '''
        total_angle = self.time * earth_angular_velocity_rps # Total radians earth rotated under bullet
        return PositionSpherical(self.r, self.θ, self.φ - total_angle, time=self.time)
    
    def gcs(self):
        '''
        Assuming the Positon is earth-fixed, returns the GSC-style coordinates
        '''
        ϕ = 90 - radians_to_degress(self.θ)
        λ = radians_to_degress(self.φ)
        return f'Latitude {ϕ}, Longitude {λ}'
    
    def google_maps_pin(self):
        ϕ = 90 - radians_to_degress(self.θ)
        λ = radians_to_degress(self.φ)
        return f'google.com/maps/search/{ϕ},+{λ}'

class Velocity:
    def __init__(self, r, θ, φ):
        self.r = r
        self.θ = θ
        self.φ = φ

def spherical_to_gcs(spherical_coordinates):
    '''
    Accepts spherical coordinates and projects them down to the surface of the earth to produce GCS coordinates
    '''
    λ = 90 - radians_to_degress(spherical_coordinates[1]) # latitude
    ϕ = radians_to_degress(spherical_coordinates[2]) # longitude
    return (λ, ϕ)

def gcs_to_spherical(gcs, h_offset_meters=0):
    '''
    Accepts GCS coordinates and turns them into spherical coordinates where the radius is just the radius of earth
    '''
    print(f'gcs_to_spherical {gcs}')
    r = earth_radius_meters + h_offset_meters
    θ = degrees_to_radians(90 - gcs[0]) # polar angle
    φ = degrees_to_radians(gcs[1]) # azimuthal angle
    print(f'gcs_to_spherical {r} {θ} {φ}')
    return PositionSpherical(r, θ, φ)

# Initialize conversion constants
meters_per_mile = 1609.34
meters_per_foot = 0.3048
seconds_per_hour = 3600
hours_per_second = 1/seconds_per_hour

# Initialize physical constants
earth_radius_miles = 3959
earth_radius_meters = earth_radius_miles * meters_per_mile
earth_rotation_period_hours = 24
earth_rotation_period_seconds = earth_rotation_period_hours * seconds_per_hour
earth_angular_velocity_rps = 2*math.pi/earth_rotation_period_seconds # radians per second
earth_mass_kg = 5.97219 * 10**(24)
G = 6.6743 * 10**(-11) # N*m^2/kg^2
g = 9.81

# Initialize problem-specific constants
bullet_initial_h_feet = 120000
#bullet_initial_h_feet = 120000 * 10

bullet_initial_h_meters = bullet_initial_h_feet * meters_per_foot
bullet_initial_r_meters = bullet_initial_h_meters + earth_radius_meters
#bullet_firing_speed_mph = 100000 # miles per hour
bullet_firing_speed_mph = 1000 # miles per hour
#bullet_firing_speed_mph = 0

bullet_firing_speed_mps = bullet_firing_speed_mph * meters_per_mile * hours_per_second
bullet_initial_horizontal_speed_mps = bullet_initial_r_meters * earth_angular_velocity_rps
#bullet_initial_horizontal_speed_mps = 0
#bullet_initial_velocity_mps = (bullet_firing_speed_mps,-bullet_initial_horizontal_speed_mps,0) # TODO: According to Kosho, the bullet has an initial radial velocity
bullet_initial_coordinates_gcs = (0, 0) # do -X for South or West, e.g. (-90, -45) for 90 S, 45 W
bullet_pos = gcs_to_spherical(bullet_initial_coordinates_gcs, bullet_initial_h_meters)

# This deserves special mention:
# The length of the spherical coordinate basis vectors depend on position:
# The radial unit vector (r̂) length does not depend on position
# The polar unit vector (θ̂) length is equal to r
# The azimuthal unit vector (ϕ̂) length is equal to r * sin(θ)
#
# Therefore to properly set velocity in spherical coordinates we must scale by the unit vector lengths at that position
bullet_vel = Velocity(0, 
                      -bullet_firing_speed_mps / bullet_pos.r,
                      bullet_initial_horizontal_speed_mps / (bullet_pos.r * math.sin(bullet_pos.θ)))

# Initialize simulation parameters
Δt = 0.1 # timestep in seconds

# Pre-process earth texture and UV-XYZ coordinates
img = plt.imread('Equirectangular_projection_SW.jpg')
img = scipy.ndimage.rotate(img, 90)
img = scipy.ndimage.zoom(img, (1/8, 1/8, 1))
img = img / 255.0
#u = -np.linspace(-np.pi/2.0, 2 * np.pi - np.pi/2.0, img.shape[0])
u = -np.linspace(np.pi, 2 * np.pi + np.pi, img.shape[0])

v = np.linspace(0, np.pi, img.shape[1])
sphere_x = earth_radius_meters * np.outer(np.cos(u), np.sin(v))
sphere_y = earth_radius_meters * np.outer(np.sin(u), np.sin(v))
sphere_z = earth_radius_meters * np.outer(np.ones(np.size(u)), np.cos(v))
       
class Bullet:
    def __init__(self, init_pos, init_vel):
        self.pos = init_pos # Fixed to space not Earth!
        self.vel = init_vel # Fixed to space not Earth!
        self.trajectory = [PositionSpherical(init_pos.r, init_pos.θ, init_pos.φ,time=0)]
        print(f'Initial position: {init_pos}')
        
    def update_pos(self):
        self.pos.r += self.vel.r * Δt
        self.pos.θ += self.vel.θ * Δt
        self.pos.φ += self.vel.φ * Δt
        
    def fire(self):
        num_iterations = 0
        max_iterations = 10000
        while self.pos.r > earth_radius_meters and num_iterations < max_iterations:
            # Gather current radial acceleration
            a = self.a()
            
            # Update position given current velocity
            self.pos.r += self.vel.r * Δt
            self.pos.θ += self.vel.θ * Δt
            self.pos.φ += self.vel.φ * Δt
            
            # Update velocity given acceleration
            self.vel.r += a * Δt
            
            # Update time
            self.pos.time += Δt
            
            # Update trajectory
            self.trajectory.append(PositionSpherical(self.pos.r, self.pos.θ, self.pos.φ, time=self.pos.time))            
            # TODO: render
            
            num_iterations += 1
        print(f'Number of iterations {num_iterations}, total time passed is {self.pos.time} seconds')
        print(f'Position {self.pos}')
                    
    def a(self):
        '''
        Returns the acceleration of the bullet (in m/s^2) in the radial direction
        '''
        return -G*earth_mass_kg/self.pos.r**2
    
    def earth_fixed_pos(self):
        '''
        Returns an earth-fixed position
        '''
        return self.pos.earth_fixed_pos()
    
    def trajectory_as_np(self):
        '''
        Returns trajectory from fixed earth not from fixed space!
        '''
        trajectory = [pos.earth_fixed_pos().cartesian() for pos in self.trajectory]
        x = np.array([pos.x for pos in trajectory])
        y = np.array([pos.y for pos in trajectory])
        z = np.array([pos.z for pos in trajectory])
        t = np.array([pos.time for pos in self.trajectory])
        return (x, y, z, t)
    
bullet = Bullet(bullet_pos, bullet_vel)

# The path the bullet takes p(t) = integral(v(t))
# v(t) = integral(a(t)) + v_initial
# a(t) = a(h(t))

# F = m_bullet * a_bullet
# G*  (m_bullet * earth_mass_kg)/r^2 = m_bullet * a_bullet
# a_bullet(h) = (G*earth_mass_kg)/(R + h)^2
# a_bullet(0) = (G*earth_mass_kg)/R^2 = g
#x = sympy.Symbol('x')
#sympy.solve()

gcs_to_spherical (0, 0)
gcs_to_spherical 6407953.06 1.5707963267948966 0.0
Initial position: (6407953.06, 1.5707963267948966, 0.0)


In [2]:
bullet.fire()
print(bullet.pos.r)

Number of iterations 868, total time passed is 86.79999999999934 seconds
Position (6371357.0642234925, 1.5647408867514887, 0.0063122741280462235)
6371357.0642234925


In [10]:
(x, y, z, t) = bullet.trajectory_as_np()
def sphere (t):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Apply the texture to the sphere and plot it
    ax.plot_surface(sphere_x, sphere_y, sphere_z, facecolors=img, linewidth=0, antialiased=False, alpha=1, zorder=1, rcount=200, ccount=200)
    #ax.plot_trisurf(sphere_x, sphere_y, sphere_z)
    
    # Plot trajectory
    ax.plot3D(x, y, z, 'red', zorder=100)
    
    ax.set_box_aspect([1, 1, 1])  # Set equal aspect ratio
    ax.set_xlabel('$X$')
    ax.set_ylabel('$Y$')
    ax.set_zlabel('$Z$')
    plt.show()
widgets.interact(sphere , t= widgets.Play(min=0,max =100));

interactive(children=(Play(value=0, description='t'), Output()), _dom_classes=('widget-interact',))

In [4]:
bullet.earth_fixed_pos()

In [5]:
bullet.pos

In [6]:
bullet.pos.time

86.79999999999934

In [7]:
bullet.pos.earth_fixed_pos().gcs()

'Latitude 0.34695115758178474, Longitude 6.4605016967227685e-15'

In [8]:
bullet.pos.earth_fixed_pos().google_maps_pin()

'google.com/maps/search/0.34695115758178474,+6.4605016967227685e-15'

In [9]:
bullet.pos.gcs()

'Latitude 0.34695115758178474, Longitude 0.3616666666666704'